In [12]:
import math
import nbimporter
from CoolProp.CoolProp import PropsSI

In [13]:
def F2K(F):
    return (F - 32) * 5/9 + 273.15

### Problem 1: Duct heat loss
A duct that is 2 ft by 5 ft in cross-section and 80 ft long carries conditioned air through the space above the hung ceiling in an office building. The air enters the duct from the air handler at 55 F and 2 in. water pressure with a flow rate of 8,000 cfm. The temperature in the space is 85 F. Determine\
a. The level of insulation needed on the sides of the duct to have a temperature rise of 1 F\
b. The drop in sensible cooling capacity due to the heat transfer to the air flow.\
c. The leakage flow rate from the duct.\
d. The total drop in capacity due to leakage and heat loss in energy and relative to the total capacity.\
Assume you are using insulation with a thermal
conductivity of kins = 0.025 Btu/hr-ft-F. Also assume a duct leakage class of 6.

In [36]:
T_i = 55 #[F]
T_o = 56
T_a = 85
V_dot = 8000
L = 80
P_i = 2
h_o = 1.47
k_ins = 0.025
Area = 2 * (2 + 5) * L

In [29]:
P_atm = 14.7 #[psi]
rho_air = PropsSI('D', 'T', F2K(T_i), 'P', 101325, 'Air') * 0.062428 # convert from kg/m3 to lb/ft3
c_p = PropsSI('C', 'T', F2K(T_i), 'P', 101325, 'Air') * 0.0002390057 # [btu/lbm-F]
m_dot = V_dot * rho_air * 60 # [lbm/hr]
Capacity = m_dot * c_p * (T_a - T_i)
print('The total capacity is:',Capacity,'Btu/hr')


The total capacity is: 266945.59015372174 Btu/hr


In [34]:
R_total = -(1/(math.log((T_a-T_o)/(T_a-T_i))*m_dot*c_p))
print(R_total)
R_o = 1/(h_o * Area)
R_ins = R_total - R_o
thinkness = R_ins * k_ins * Area * 12
print(thinkness)

0.0033149648482099533
0.9097465563454832


In [ ]:
Q_cap_drop = m_dot * c_p *(T_o - T_i)
print (Q_cap_drop)


8898.186338457392


In [40]:
C_L = 6/100
V_dot_leak = C_L * (Area) * P_i**0.65
Q_leak = V_dot_leak * 60 * rho_air * c_p *(T_a - T_i)
print (Q_leak)

3518.6132526429956


In [41]:
precent_cap = (Q_cap_drop + Q_leak)/Capacity * 100
print(precent_cap)

4.651434617799874
